In [6]:
import numpy as np
import pandas as pd

import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
import tkinter.font as font
import os

In [7]:
Questions = {
    'مخرجات التعليم المستهدفة': 
    [
        [
            'المقرر له اهداف واضحة ومعلنة​',
            'المقرر ينمي القدرة على التفكير والابتكار', 
            'المقرر يزودني بالمعرفة المفيدة والفهم المتعلق بالموضوع',
            'يجمع المقرر بين الجانب النظري والتطبيقي',
        ], 0
    ],
    
    'المقرر والكتاب':
    [
        [
            'محتويات المقرر تتناسب مع الوقت المخصص',
            'تتوافر المراجع بمكتبة الكلية أو بأحد الصور الإلكترونية الأخرى'
        ], 0
    ],
    
    'وسائل وطرق التقييم':
    [
        [
            'الامتحانات تعكس محتوى المقرر وتشتمل على الجوانب النظرية والعملية',
            'تعتبر اللغة المستخدمة في الامتحانات واضحة ومفهومة',
            'لا تتضمن الامتحانات اخطاء مطبعية'
        ], 0
    ],
    
    'المحاضر':
    [
        [
            'قام المحاضر بشرح محتويات المقرر وطرق التقييم عند بدء الدراسة',
            'مدى وضوح أسلوب عرض المحاضر للمادة العلمية للمقرر',
            'التزم المحاضر بحضور المحاضرات في مواعيدها وطبقا للجدول المعلن',
            'اهتم المحاضر بالربط بين المادة العلمية والتطبيقات العملية لموضوعات المقرر',
            'استخدم المحاضر وسائل تعليمية متنوعة ومناسبة لشرح المقرر',
            'شجع المحاضر علي المناقشة والتعليق وإلقاء الأسئلة والرد عليها',
            'مدى قدرة المحاضر على إدارة وضبط المحاضرة',
            'يساوى المحاضر في التعامل مع جميع الطلاب',
            'يلتزم المحاضر بالتواجد في مكتبه في الساعات المكتبية للتواصل مع الطلاب',
            'هل ترغب في دراسة مقررات أخرى مع نفس المحاضر مستقبلا؟'
        ], 0
    ],
    
    'الهيئة المعاونة(معيد المقرر)':
    [
        [
        'التزم المعيد بحضور الفصول والمعامل في مواعيدها',
        'يشجع المعيد على العمل في مجموعات طلابية أثناء التمارين والعملي',
        'يساوى المعيد في التعامل مع جميع الطلاب',
        'يتناول المعيد موضوعات تغطي الجانب العملي والتطبيقي للمقرر',
        'يستثمر المعيد وقت الفصول والمعامل في الشرح والمناقشة ويوفر التطبيقات الكافية',
        'هل ترغب في دراسة مقررات أخرى مع نفس المعيد مستقبلا؟'
        ], 0
    ]
}

In [8]:
def analyze(data, file_name):
    for cat in Questions:
        Questions[cat][1] = 0

        
    data_formated = []

    for col in data.columns[6:]:
        total = data[col].count()
        question = col
        strongly_agree = (data[col].astype(str).str.strip() == 'موافق بشدة').sum()
        agree = (data[col].astype(str).str.strip() == 'موافق').sum()
        maybe = (data[col].astype(str).str.strip() == 'إلى حد ما').sum()
        disagree = (data[col].astype(str).str.strip() == 'غير موافق').sum()
        strongly_disagree = (data[col].astype(str).str.strip() == 'غير موافق بشدة').sum()

        nums = [question, strongly_agree, agree, maybe, disagree, strongly_disagree, 0]
        data_formated.append(nums)

        perc = np.array([0, strongly_agree, agree, maybe, disagree, strongly_disagree])
        perc = perc / total
        perc = np.round(perc*100, 1)

        weights = np.array([3, 2, 1, 0, -1])
        count = nums[1:-1]
        result = weights.dot(count) / sum(count) / max(weights) * 100
        result = np.round(result, 1)
        
        for cat in Questions:
            if col.strip() in Questions[cat][0]:
                Questions[cat][1] += result
                
        

        perc = perc.tolist()
        perc.append(result)
        perc = [x.replace('.0', '') + '%' for x in np.array(perc).astype('str')]

        data_formated.append(perc)

    df = pd.DataFrame(data=data_formated, columns=['السؤال',
                                                   'موافق بشدة',
                                                   'موافق',
                                                   'إلى حد ما',
                                                   'غير موافق', 
                                                   'غير موافق بشدة', 
                                                   'النتيجة'
                                                  ])
    df = df.replace(0, '')
    df = df.replace('0.0', '')
    df = df.replace('0%', '')
    df = df.replace('nan%', '')
    
    name = os.path.basename(file_name)
    df.to_excel('2'+name, index=False)
    

    final_results = []
    for cat in Questions:
        total_result = np.round(Questions[cat][1] / len(Questions[cat][0]), 1)
        final_results.append([cat, total_result])
    final_results.append(['النتيجة', np.round(np.average([x[1] for x in final_results]), 1)])
    
    final_results = [[x, str(y) + '%'] for x, y in final_results]
    df_results = pd.DataFrame(final_results, columns = ['السؤال', 'النسبة'])
    display(df_results)

In [9]:
def selectFile():
    file_name = filedialog.askopenfilename(initialdir="/", title="Select file")
    
    if not file_name:
        return
    
    try:
        data = pd.read_excel(file_name)                                 
        analyze(data, file_name)
        messagebox.showinfo("Information", "Done")
    except:
        messagebox.showerror("Error", "Failed")
        return

In [10]:
window = tk.Tk()
window.geometry("400x200")

button = tk.Button(window, bg='#0052cc', fg='#ffffff', text="Select file",
                   command = selectFile)
button['font'] = font.Font(size=15)

button.place(relx=0.5, rely=0.5, anchor=CENTER)

window.mainloop()